## Basic Importations

In [27]:
from crypto_price import approval_program, clear_state_program
import base64
import os

from algosdk import mnemonic
from algosdk.future import transaction
from algosdk.v2client import algod
from dotenv import load_dotenv
from pyteal import *

load_dotenv()

True

### Purestake connection

In [28]:
algod_address = os.getenv('PURESTAKE_API')

### Add Token

In [29]:
algod_token = os.getenv('PURESTAKE_TOKEN')
headers = {
    "X-API-KEY": algod_token
}
algod_client = algod.AlgodClient(algod_token, algod_address, headers)

# Setup the wallet

In [30]:
passphrase = os.getenv('WALLET_CREATOR')  # Put your passphrase here

private_key = mnemonic.to_private_key(passphrase)
sender = mnemonic.to_public_key(passphrase)
print("Sender address: {}".format(sender))

account_info = algod_client.account_info(sender)
print("Account balance: {} microAlgos".format(account_info.get('amount')))

Sender address: DGFE25IFWLNJ2AQ654PJH3RIEXIDOVKDJS5HYNABO4DFZECNISOVDX5RCE
Account balance: 29998000 microAlgos


## Setup Smart contract data scheme

In [31]:
local_ints = 0
local_bytes = 0
global_ints = 60
global_bytes = 1

global_schema = transaction.StateSchema(global_ints, global_bytes)
local_schema = transaction.StateSchema(local_ints, local_bytes)

### Basic Txn config

In [32]:
params = algod_client.suggested_params()
on_complete = transaction.OnComplete.NoOpOC.real

### Helper function that waits for a given txid to be confirmed by the network

In [33]:
def wait_for_confirmation(client, txid):
    last_round = client.status().get('last-round')
    txinfo = client.pending_transaction_info(txid)
    while not (txinfo.get('confirmed-round') and txinfo.get('confirmed-round') > 0):
        print("Waiting for confirmation...")
        last_round += 1
        client.status_after_block(last_round)
        txinfo = client.pending_transaction_info(txid)
    print("Transaction {} confirmed in round {}.".format(
        txid, txinfo.get('confirmed-round')))
    return txinfo

# Compile pyteal code

In [34]:
approval_program_source_code = compileTeal(
    approval_program(), Mode.Application, version=4)
clear_state_program_source_code = compileTeal(
    clear_state_program(), Mode.Application, version=4)


def compile_program(source_code):
    compile_response = algod_client.compile(source_code)
    return base64.b64decode(compile_response['result'])
    

did_approval_program = compile_program(approval_program_source_code)
did_clear_state_program = compile_program(clear_state_program_source_code)


### Create Txn to create the Dapp

In [35]:
txn = transaction.ApplicationCreateTxn(sender, params, on_complete, did_approval_program, did_clear_state_program,
                                       global_schema, local_schema)

### Sign transaction

In [36]:
signed_txn = txn.sign(private_key)
tx_id = signed_txn.transaction.get_txid()

print(tx_id)

IZY5E2UW5PNPHABXQMR4ZNBYQHKGKXL5TMXED5JSGAHEFUKG5NPQ


### Send txn

In [37]:
algod_client.send_transactions([signed_txn])

'IZY5E2UW5PNPHABXQMR4ZNBYQHKGKXL5TMXED5JSGAHEFUKG5NPQ'

### Await confirmation

In [38]:
wait_for_confirmation(algod_client, tx_id)

Waiting for confirmation...
Transaction IZY5E2UW5PNPHABXQMR4ZNBYQHKGKXL5TMXED5JSGAHEFUKG5NPQ confirmed in round 19182067.


{'application-index': 62242291,
 'confirmed-round': 19182067,
 'global-state-delta': [{'key': 'Q3JlYXRvcg==',
   'value': {'action': 1,
    'bytes': 'GYpNdQWy2p0CHu8ek+4oJdA3VUNMunw0AXcGXJBNRJ0='}}],
 'pool-error': '',
 'txn': {'sig': '/Cr8Bk8T9CSP7xU3BSGDVisNPUJLhk6/DWnH9D//O+E153IkJXBiHUv/NjPjN+Xlyyw/M7dqc3rTKSjyDvpeDg==',
  'txn': {'apap': 'BCADARAAJgEHQ3JlYXRvcjEYJBJAAz0xGSISQAMuMRmBBBJAAx4xGYEFEkADDjEZgQMSQAL+NhoAgA11cGRhdGVfc2xvdF8xEkACPjYaAIANdXBkYXRlX3Nsb3RfMhJAAX82GgCADXVwZGF0ZV9zbG90XzMSQAC8NhoAgA11cGRhdGVfc2xvdF80EkAAAQAxAChkEkQxGyMSRIADRU9TNhoBF2eABENBS0U2GgIXZ4ADU1RYNhoDF2eABEZMT1c2GgQXZ4ADTFJDNhoFF2eAA09ORTYaBhdngANCVFQ2GgcXZ4ADS1NNNhoIF2eAA01LUjYaCRdngANFTko2GgoXZ4ADQlNWNhoLF2eAA1FOVDYaDBdngANBTVA2Gg0XZ4ADS0RBNhoOF2eAA1hFQzYaDxdnIkMxAChkEkQxGyMSRIAESEJBUjYaARdngARCVENCNhoCF2eABVRIRVRBNhoDF2eABEVHTEQ2GgQXZ4ADSUNQNhoFF2eAA0VUQzYaBhdngAVNSU9UQTYaBxdngANYVFo2GggXZ4ADSE5UNhoJF2eAA1hNUjYaChdngARBQVZFNhoLF2eAA0xFTzYaDBdngARLTEFZNhoNF2eABEdBTEE2Gg4XZ4ADR1JUNhoPF2ciQzEAKGQSRDEbIx

### Display results

In [39]:
transaction_response = algod_client.pending_transaction_info(tx_id)
app_id = transaction_response['application-index']
print("Created new app-id: ", app_id)

Created new app-id:  62242291
